# load the data

In [18]:
from pprint import pprint
import json, os

# load a scrapped dataset
# json_path = r'C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\dFBA\dFBApy\scraping\SABIO\scraped_content\sabio_proccessed.json' # this is for some reason not functional
json_path = 'sabio_proccessed.json'

with open(json_path) as data:
    sabio = json.load(data)
# sabio = json.load(open(json_path, 'r'))
# pprint(sabio, depth = 1)

# parse and consolidate the data

In [84]:
from scipy.constants import micro, nano, milli, minute
import re

def isnumber(num):
    try:
        string = str(num)
        string = string.strip()
        if re.sub('([0-9.])', '',string) == '':
            return True
    except:
        return False
    
def average(num_1, num_2 = None):
    if isnumber(num_1): 
        if isnumber(num_2):
            numbers = [float(num_1), float(num_2)]
            average = sum(numbers) / len(numbers)
            return average
        else:
            return float(num_1)
    elif isnumber(num_2): 
        return float(num_2)
    elif type(num_1) is list:
        average = sum(num_1) / len(num_1)
        return average
    else:
        return None
    
def _determine_parameter_value(unit, original_value):
    # parse the unit
    numerator = ''
    denominator = ''
    term = ''
    skips = 0
    next_denominator = False
    for index in range(len(unit)):
        if skips > 0:
            skips -= 1 
            continue
            
        ch = unit[index]
        term += ch
        
        # parse the unit characters
        if len(unit) == 1:
            numerator += term
            term = ''
            break
        if index+1 == len(unit)-1:
            if next_denominator:
                denominator += term
                denominator += unit[index+1]
            else:
                numerator += term
                numerator += unit[index+1]
            term = ''
            break
        if unit[index+1] == '^':
            if unit[index+2:index+6] == '(-1)':
                denominator += term 
                skips = 5
                term = ''
            else:
                print(unit, term)
        elif unit[index+1] == '/':
            numerator += term
            term = ''
            skips = 1
            next_denominator = True
            
    if term != '':
        print(unit, term)
    unit_dic = {
        'numerator':numerator,
        'denominator': denominator
    }
    
    # determine the mathematically equivalent value in base units
    value = original_value
    for group in unit_dic:
        term = unit_dic[group]
        if re.search('mg|mM|mmol', term):
            if group == 'numerator':
                value *= milli
            else:
                value /= milli                
        elif re.search('ng|nM|nmol', term):
            if group == 'numerator':
                value *= nano
            else:
                value /= nano   
        elif re.search('µg|µmol|µM|u00b5g|u00b5mol|u00b5M|U\+00B5g|U\+00B5mol|U\+00B5M', term):
            if group == 'numerator':
                value *= micro
            else:
                value /= micro   
        elif re.search('min', unit):
            if group == 'numerator':
                value *= minute
            else:
                value /= minute  
    return value
            
# parse the values          
unit_prevalence = {}
consolidated_sabio = {}
missing_content = {}
pars = {}
for enzyme in sabio:
#     print('\n', enzyme, '\n', '='*2*len(enzyme))
    consolidated_sabio[enzyme] = {}
    for reaction in sabio[enzyme]:   
        consolidated_sabio[enzyme][reaction] = {}
        for entry_id in sabio[enzyme][reaction]:  
            for par in sabio[enzyme][reaction][entry_id]['Parameters']:
                if par in pars:
                    pars[par] += 1
                elif par not in pars:
                    pars[par] = 1
                param = sabio[enzyme][reaction][entry_id]['Parameters'][par]
                if 'type' in param:
                    if 'unit' in param:
#                         if param['type'] == 'concentration':
                        if not param['species'] == 'Enzyme':
                            initial_value = param['start val.']
                            final_value = param['end val.']
                            value = average(initial_value, final_value)
                            if value:
                                unit = param['unit']

                                if unit in unit_prevalence:
                                    unit_prevalence[unit] += 1
                                else:
                                    unit_prevalence[unit] = 1

                                # determine the value


                                # standardize the units to base units
                                print('here', unit, value)
                                _determine_parameter_value(unit, value)
                                
                elif 'unit' in param:
                    if param['unit'] == '°C':
                        temperature = param['start value']

print('parameters', )
pprint(pars)
print('\n\nunits', )
pprint(unit_prevalence)
# print('\nmissing_content', )
# pprint(missing_content)

here mM 0.5700000000000001
{'numerator': 'mM', 'denominator': ''} 0.5700000000000001 0.0005700000000000001
here mM 0.6
{'numerator': 'mM', 'denominator': ''} 0.6 0.0006
here µM 50.0
{'numerator': 'µM', 'denominator': ''} 50.0 4.9999999999999996e-05
here µM 0.047
{'numerator': 'µM', 'denominator': ''} 0.047 4.7e-08
here µM 360.0
{'numerator': 'µM', 'denominator': ''} 360.0 0.00035999999999999997
here µM 0.34
{'numerator': 'µM', 'denominator': ''} 0.34 3.4000000000000003e-07
here U/mg 68.0
{'numerator': 'U', 'denominator': 'mg'} 68.0 68000.0
here mM 0.12000000000000001
{'numerator': 'mM', 'denominator': ''} 0.12000000000000001 0.00012000000000000002
here M 0.035
{'numerator': 'M', 'denominator': ''} 0.035 0.035
here µM 100.0
{'numerator': 'µM', 'denominator': ''} 100.0 9.999999999999999e-05
here µM 350.0
{'numerator': 'µM', 'denominator': ''} 350.0 0.00035
here µM 20.0
{'numerator': 'µM', 'denominator': ''} 20.0 1.9999999999999998e-05
here µM 5.8
{'numerator': 'µM', 'denominator': ''} 5.

{'numerator': 'mM', 'denominator': ''} 15.25 0.01525
here s^(-1) 4.65
{'numerator': '', 'denominator': 's'} 4.65 4.65
here M^(-1)*s^(-1) 1430.0
{'numerator': '', 'denominator': 'M*s'} 1430.0 1430.0
here mM 3.25
{'numerator': 'mM', 'denominator': ''} 3.25 0.0032500000000000003
here - 7.1
{'numerator': '-', 'denominator': ''} 7.1 7.1
here - 10.0
{'numerator': '-', 'denominator': ''} 10.0 10.0
here mM 7.0
{'numerator': 'mM', 'denominator': ''} 7.0 0.007
here mM 15.25
{'numerator': 'mM', 'denominator': ''} 15.25 0.01525
here s^(-1) 5.14
{'numerator': '', 'denominator': 's'} 5.14 5.14
here M^(-1)*s^(-1) 526.0
{'numerator': '', 'denominator': 'M*s'} 526.0 526.0
here mM 9.77
{'numerator': 'mM', 'denominator': ''} 9.77 0.00977
here - 19.0
{'numerator': '-', 'denominator': ''} 19.0 19.0
here - 9.1
{'numerator': '-', 'denominator': ''} 9.1 9.1
here mM 30.0
{'numerator': 'mM', 'denominator': ''} 30.0 0.03
here mM 3.7
{'numerator': 'mM', 'denominator': ''} 3.7 0.0037
here s^(-1) 0.53
{'numerator':